In [1]:
import akshare as ak 

import pandas as pd
import numpy as np

import os 
from datetime import datetime
from tqdm import tqdm

### load the stock list

In [2]:
# load the hongli list
hl = pd.read_csv('../data/input/hongli_list_20251213.csv')
hl['code'] = hl['code'].astype(str).str.zfill(6)

hl_stocks = hl['code'].tolist()
print(len(hl_stocks))

# load the honglidbo stock list
hldb = pd.read_csv('../data/input/honglidibo_list_20251213.csv')
hldb['code'] = hldb['code'].astype(str).str.zfill(6)

hldb_stocks = hldb['code'].tolist()
print(len(hldb_stocks))

# convert stock to symbols
hl_symbols = [f"sh{code}" if code.startswith("6") 
                else f"bj{code}" if code.startswith("9") 
                else f"sz{code}"
                for code in hl_stocks]
hldb_symbols = [f"sh{code}" if code.startswith("6")
                else f"bj{code}" if code.startswith("9") 
                else f"sz{code}"
                for code in hldb_stocks]

100
50


In [3]:
query_date = '20251226'

# zhongzheng hongli
hl_values = []
for stock in hl_stocks:
    value = pd.read_csv(f"../data/processed/stock-valuation/{query_date}/stock_valuation_{stock}_{query_date}.csv")
    value['code'] = stock.zfill(6)
    value_current = value.iloc[-1]
    hl_values.append(value_current)

hl_values = pd.DataFrame(hl_values, columns=value.columns)
hl_values = pd.merge(hl_values, hl[['code', 'name', 'weight']], on='code', how='left', validate='1:1')
print(hl_values.shape)
os.makedirs(f'../data/output/etf/{query_date}', exist_ok=True)
hl_values.to_csv(f'../data/output/etf/{query_date}/hl_valuation_{query_date}.csv', index=False)

# zhongzheng hongli dibo
hldb_values = []
for stock in hldb_stocks:
    value = pd.read_csv(f"../data/processed/stock-valuation/{query_date}/stock_valuation_{stock}_{query_date}.csv")
    value['code'] = stock.zfill(6)
    value_current = value.iloc[-1]
    hldb_values.append(value_current)

hldb_values = pd.DataFrame(hldb_values, columns=value.columns)
hldb_values = pd.merge(hldb_values, hldb[['code', 'name', 'weight']], on='code', how='left', validate='1:1')
print(hldb_values.shape)
hldb_values.to_csv(f'../data/output/etf/{query_date}/hldb_valuation_{query_date}.csv', index=False)

(100, 20)
(50, 20)


### calculate etf values

In [5]:
# assuming a total 100 investment
# earning yield = sum(shares * eps) = sum(invest/ price * eps) / 100
hl_values['earning_yield'] = hl_values['weight'] / hl_values['close'] * hl_values['eps_ttm']
# net asset yield = sum(shares * bps) = sum(invest / price * bps) / 100
hl_values['net_asset_yield'] = hl_values['weight'] / hl_values['close'] * hl_values['bps_ttm']

hl_total_investment = hl_values['weight'].sum()
hl_earning_yield = hl_values['earning_yield'].sum() / hl_total_investment
hl_net_asset_yield = hl_values['net_asset_yield'].sum() / hl_total_investment
hl_weited_roe = hl_earning_yield / hl_net_asset_yield

print(f"中证红利：")
print(f"earning yield: {hl_earning_yield:.2f}")
print(f"net asset yield: {hl_net_asset_yield:.2f}")
print(f"weighted roe: {hl_weited_roe:.2f}")

中证红利：
earning yield: 0.09
net asset yield: 1.10
weighted roe: 0.08


In [6]:
# assuming a total 100 investment
# earning yield = sum(shares * eps) = sum(invest/ price * eps) / 100
hldb_values['earning_yield'] = hldb_values['weight'] / hldb_values['close'] * hldb_values['eps_ttm']
# net asset yield = sum(shares * bps) = sum(invest / price * bps) / 100
hldb_values['net_asset_yield'] = hldb_values['weight'] / hldb_values['close'] * hldb_values['bps_ttm']

hldb_total_investment = hldb_values['weight'].sum()
hldb_earning_yield = hldb_values['earning_yield'].sum() / hldb_total_investment
hldb_net_asset_yield = hldb_values['net_asset_yield'].sum() / hldb_total_investment
hldb_weited_roe = hldb_earning_yield / hldb_net_asset_yield

print(f"红利低波：")
print(f"earning yield: {hldb_earning_yield:.2f}")
print(f"net asset yield: {hldb_net_asset_yield:.2f}")
print(f"weighted roe: {hldb_weited_roe:.2f}")

红利低波：
earning yield: 0.12
net asset yield: 1.22
weighted roe: 0.09


### send emails

In [7]:
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.header import Header
from smtplib import SMTP_SSL
from datetime import datetime

import pandas as pd

import os 
from dotenv import load_dotenv

In [8]:
def send_mail(receiver='', mail_title='', mail_content=''):
    # ssl login
    smtp = SMTP_SSL(host_server)
    # set_debuglevel() for debug, 1 enable debug, 0 for disable
    # smtp.set_debuglevel(1)
    smtp.ehlo(host_server)
    smtp.login(sender_mail, sender_passcode)

    # construct message
    msg = MIMEText(mail_content, "plain", 'utf-8')
    msg["Subject"] = Header(mail_title, 'utf-8')
    msg["From"] = sender_mail
    msg["To"] = receiver
    smtp.sendmail(sender_mail, receiver, msg.as_string())
    smtp.quit()

In [9]:
load_dotenv()
today = datetime.now().strftime('%Y%m%d')

# qq mail sending server
host_server = os.getenv("SMTP_SERVER")
sender_mail = os.getenv("SENDER_EMAIL")
sender_passcode = os.getenv("EMAIL_AUTH_CODE")

# receiver mail
receiver = os.getenv("RECEIVER_EMAIL")
# mail title
mail_title = f'ETF Intrinsic Value by {query_date}'
# mail contents
mail_content = f'中证红利 intrinsic value by {query_date} are: \n' + \
f"earning yield: {hl_earning_yield:.4f} \n" + \
f"net asset yield: {hl_net_asset_yield:.2f} \n" + \
f"weighted roe: {hl_weited_roe:.2f} \n" + \
f'红利低波 intrinsic value by {query_date} are: \n' + \
f"earning yield: {hldb_earning_yield:.4f} \n" + \
f"net asset yield: {hldb_net_asset_yield:.2f} \n" + \
f"weighted roe: {hldb_weited_roe:.2f} \n"

send_mail(receiver=receiver,mail_title=mail_title,mail_content=mail_content)
print('Email sent successfully.')

Email sent successfully.
